In [422]:
def pPrise(letter, memory, kp):
    length = len(memory)
    if length > 0:
        f = memory.count(letter)/length
        return pow(kp/(kp+f), 2)
    return 0

In [423]:
def pDepot(letter, memory, kd):
    length = len(memory)
    if length > 0:
        f = memory.count(letter)/length
        return pow(f/(kd+f), 2)
    return 0

La classe de l'Agent contenant sa position et sa mémoire (ainsi que ses différents mouvements en bonus)

In [504]:
class Agent:
    def __init__(self, x, y, t):
        self.x = x
        self.y = y
        self.t = t
        self.memory = []
        self.mouvements = []
        
    def __str__(self):
        return "("+str(self.x)+","+str(self.y)+","+str(memory)+")"
    
    def copyss(self, agent):
        self.x = agent.x
        self.y = agent.y
        self.t = agent.t
        self.memory = agent.memory
    
    def full(self):
        return len(self.memory) >= self.t
    
    def add(self,letter):
        self.memory.append(letter)
        if self.full():
            self.memory.pop(0)

Une classe pas très utile dans l'absolu mais si nous souhaitons ajouter des informations dans la différentes cases, cette classe est primordiale.

In [505]:
class Case:
    def __init__(self,symbol):
        self.symbol = symbol
    def __str__(self):
        return self.symbol
    def getSymbole(self):
        return self.symbol

Principale classe qui regroupe l'initialisation de la matrice et ses agents, leur déplacement et la gestion de leur mémoire. Sans parler de l'algorithme qui gère le tri collectif (voir play).

In [543]:
import copy
import numpy

class TriCollectif:
    def __init__(self, n, m, nA, nB, nAgents, mov, kd, kp, t):
        self.n = n
        self.m = m
        self.nA = nA
        self.nB = nB
        self.nAgents = nAgents
        self.mov = mov
        self.kp = kp
        self.kd = kd
        self.t = t
        self.matrice = [[Case('0') for y in range(0,m)] for x in range(0,n)]
        self.matriceAgents = [[Case('0') for y in range(0,m)] for x in range(0,n)]
        self.agents = []
    
    def afficher(self):
        [print('|'.join([self.matrice[x][y].symbol for y in range(0,m)])) for x in range(0,n)]
        print()
    
    def afficherAgents(self):
        self.matriceAgents = [[Case('0') for y in range(0,m)] for x in range(0,n)]
        for agent in self.agents:
            self.matriceAgents[agent.x][agent.y].symbol = 'x'
        [print('|'.join([self.matriceAgents[x][y].symbol for y in range(0,m)])) for x in range(0,n)]
    
    def afficherMemory(self):
        for agent in self.agents:
            print(agent.memory, " / ", agent.mouvements)
            print()
        
    def init(self):
        cptA = self.nA
        cptB = self.nB
        while cptA > 0:
            x = randrange(self.n)
            y = randrange(self.m)
            if random.getrandbits(1) and self.matrice[x][y].symbol=='0':
                self.matrice[x][y].symbol = 'a'
                cptA -= 1
        while cptB > 0:
            x = randrange(self.n)
            y = randrange(self.m)
            if random.getrandbits(1) and self.matrice[x][y].symbol=='0':
                self.matrice[x][y].symbol = 'b'
                cptB -= 1
        
        self.agents = []
        for i in range(0, self.nAgents):
            x = randrange(n)
            y = randrange(m)
            self.agents.append(Agent(x,y,self.t))
    
    def verif(self, agent):
        if agent.x >= 0 and agent.x < self.n and agent.y > 0 and agent.y < self.m:
            return 1
        else:
            return 0
        
    def direct(self, agent, i):
        method_name='dir_'+str(i)
        method=getattr(self,method_name,lambda :'Invalid')
        newAgent = Agent(0,0,0)
        newAgent.x = agent.x
        newAgent.y = agent.y
        method(newAgent)
        if self.verif(newAgent):
            agent.x = newAgent.x
            agent.y = newAgent.y
            return 1
        else:
            return 0
    
    def dir_0(self, agent):
        agent.x = agent.x - self.mov
    
    def dir_1(self, agent):
        agent.y = agent.y + self.mov
    
    def dir_2(self, agent):
        agent.x = agent.x + self.mov
        
    def dir_3(self, agent):
        agent.y = agent.y - self.mov
    
    def move(self, agent):
        direction = randrange(3)
        accepted = self.direct(agent, direction)
        while not accepted:
            direction = randrange(3)
            accepted = self.direct(agent, direction)
        
        agent.mouvements.append(direction)
        
    def modif(self, agent):
        letter = self.matrice[agent.x][agent.y].symbol
        if letter == '0':
            pDA = pDepot('a', agent.memory, self.kd)
            pDB = pDepot('b', agent.memory, self.kd)
            choix = numpy.random.choice(numpy.arange(0, 3), p=[(1 - pDA - pDB), pDA, pDB])
            if choix == 1:
                agent.memory.remove('a')
                self.matrice[agent.x][agent.y].symbol = 'a'
            elif choix == 2:
                agent.memory.remove('b')
                self.matrice[agent.x][agent.y].symbol = 'b'
            else:
                agent.add(letter)
        else:
            pPletter = pPrise(letter, agent.memory, self.kp)
            choix = numpy.random.choice(numpy.arange(0, 2), p=[(1 - pPletter), pPletter])
            if choix == 1:
                agent.add(letter)
                self.matrice[agent.x][agent.y].symbol = '0'
        #print(agent.memory)
                
        
    def play(self):
        steps = 2000
        for i in range(0, steps):
            for agent in self.agents:
                self.move(agent)
                self.modif(agent)

Intialisation des paramètres comme donnés sur l'énoncé.

In [547]:
n = 50
m = 50
nA = 200
mB = 200
nAgents = 20
movMax = int((min([n,m])/2)/nbAgents)
t = 10
mov = 1
kp = 0.1
kd = 0.3
t=10

Paramétrage du tri collectif

In [548]:
tc = TriCollectif(n, m, nA, nB, nAgents, mov, kd, kp, t)

Initialisation du tri collectif et lancement de son algorithme.
Affichage des positions des agents, de la matrice avant puis après le lancement de l'algo.

In [549]:
tc.init()
tc.afficherAgents()
print()
tc.afficher()
tc.play()
print()
#tc.afficherMemory()
print()
tc.afficher()


0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
x|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|x|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|x|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|